# Capstone Project by Nicolas de Wergifosse

---

### Week 1

#### Introduction/Business Problem :

Who : Am an investor in the health sector.

What : I plan to buy a building and open a new crossfit center.

Where : In Paris.

When : As soon as I am certain about the neighborhood where to locate it.

Why : Because it's a new trend that clearly exploded recently and customers are looking for having a center closed to home.



 #### Description of the data approach to figure out where is the best spot :
Step 1 : Importing the necessary libraries.

Step 2 : Connect to the Foursquare API using personal credentials.

Step 3 : Build the dataframe.

Step 4 : Folium visualization of the of our data set area.

Step 5 : Using Foursquare, let's find info on the crossfit centers in the neighborhoods.

Step 6 : Build a dataframe based on Foursquare results and clean up the data.

Step 7 : Extract the number of crossfit centers per "Arrondissement" and visualize it.

Step 8 : Decision criteria.

Step 9 : Conclusion.

### Week 2

##### Step 1 : Import all the necessary libraries :

In [1]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


##### Step 2 : Connect to the Foursquare API using personal credentials

In [3]:
CLIENT_ID = 'JQTBOUDNJ5X50MOGXTW3JPHPAGRMSSKF40KKF5HWZLIWOXCO' # your Foursquare ID
CLIENT_SECRET = 'I0WCZEH2V0MQMHGHR404JEPXORYA5CGSUM251OIC5V15PLBW' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JQTBOUDNJ5X50MOGXTW3JPHPAGRMSSKF40KKF5HWZLIWOXCO
CLIENT_SECRET:I0WCZEH2V0MQMHGHR404JEPXORYA5CGSUM251OIC5V15PLBW


##### Step 3 : Build the dataframe

In [87]:
paris = {'Borough': ['75001','75002','75003','75004','75005','75006','75007','75008','75009','75010','75011','75012','75013','75014','75015','75016','75017','75018','75019','75020'],
        'Neighborhood': ['Louvre','Bourse','Temple','Hotel de Ville','Panthéon','Luxembourg','Palais Bourbon','Elysée','Opéra','Saint Laurent','Popincourt','Reuilly','Gobelins','Observatoire','Vaugirard','Passy','Batignolles','Montmartre','Buttes Chaumont','Ménilmontant'],
        'Density': [9041,21006,29956,16966,23359,19734,13235,9457,27251,31754,40827,22345,25625,24821,27712,20921,29724,32875,27342,32702],
        'Pricem2': [13681,12274,12919,13655,13107,15416,14650,13160,11623,10439,10779,10041,9969,10702,10783,11903,11204,9990,8895,9127],
        'Latitude': [48.859200,48.865500,48.863700,48.860100,48.8448,48.849300,48.856500,48.876300,48.871800,48.870900,48.857400,48.841200,48.832200,48.833100,48.841200,48.863700,48.883500,48.892500,48.881700,48.864600],
        'Longitude': [2.341700,2.342600,2.361500,2.350700,2.347100,2.330000,2.321000,2.318300,2.339900,2.356100,2.379500,2.387600,2.356100,2.326400,2.300300,2.276900,2.321900,2.344400,2.382200,2.398400]
        }

df = pd.DataFrame(paris, columns = ['Borough','Neighborhood','Density','Pricem2','Latitude','Longitude'])

print (df)

   Borough     Neighborhood  Density  Pricem2  Latitude  Longitude
0    75001           Louvre     9041    13681   48.8592     2.3417
1    75002           Bourse    21006    12274   48.8655     2.3426
2    75003           Temple    29956    12919   48.8637     2.3615
3    75004   Hotel de Ville    16966    13655   48.8601     2.3507
4    75005         Panthéon    23359    13107   48.8448     2.3471
5    75006       Luxembourg    19734    15416   48.8493     2.3300
6    75007   Palais Bourbon    13235    14650   48.8565     2.3210
7    75008           Elysée     9457    13160   48.8763     2.3183
8    75009            Opéra    27251    11623   48.8718     2.3399
9    75010    Saint Laurent    31754    10439   48.8709     2.3561
10   75011       Popincourt    40827    10779   48.8574     2.3795
11   75012          Reuilly    22345    10041   48.8412     2.3876
12   75013         Gobelins    25625     9969   48.8322     2.3561
13   75014     Observatoire    24821    10702   48.8331     2.

##### Step 4 : Folium visualization of the of our data set area

In [33]:
map_paris = folium.Map(location=[48.8534,2.3488],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_paris)
map_paris

##### Step 5 : Using Foursquare, let's find info on the crossfit centers in the neighborhoods

In [38]:
address = '75000, Paris, France'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

48.8566969 2.3514616


In [39]:
search_query = 'crossfit'
radius = 10000
print(search_query + ' .... OK!')

crossfit .... OK!


In [89]:
# Get the Foursquare URL :
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=JQTBOUDNJ5X50MOGXTW3JPHPAGRMSSKF40KKF5HWZLIWOXCO&client_secret=I0WCZEH2V0MQMHGHR404JEPXORYA5CGSUM251OIC5V15PLBW&ll=48.8566969,2.3514616&v=20180604&query=crossfit&radius=10000&limit=30'

In [90]:
# Extract results from Foursquare DB :
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eaf0b96949393001b19ba96'},
 'response': {'venues': [{'id': '53e796e1498ed133e2e63026',
    'name': 'CrossFit Lutèce',
    'location': {'address': '4 Rue Jules Simon',
     'lat': 48.84088808067835,
     'lng': 2.292198728602586,
     'labeledLatLngs': [{'label': 'display',
       'lat': 48.84088808067835,
       'lng': 2.292198728602586}],
     'distance': 4684,
     'postalCode': '92015',
     'cc': 'FR',
     'city': 'Paris',
     'state': 'Île-de-France',
     'country': 'France',
     'formattedAddress': ['4 Rue Jules Simon', '92015 Paris', 'France']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588530081',
    'hasPerk': False},
   {'id': '56a7c54b498e2517c6b9760b',
    'name': 'CrossFit Original Addicts II',

##### Step 6 : Build a dataframe based on Foursquare results and clean up the data

In [42]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,53e796e1498ed133e2e63026,CrossFit Lutèce,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1588524588,False,4 Rue Jules Simon,48.840888,2.292199,"[{'label': 'display', 'lat': 48.84088808067835...",4684,92015,FR,Paris,Île-de-France,France,"[4 Rue Jules Simon, 92015 Paris, France]",NaN,NaN
1,56a7c54b498e2517c6b9760b,CrossFit Original Addicts II,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1588524588,False,37 Rue Bréguet,48.858407,2.375695,"[{'label': 'display', 'lat': 48.85840655301172...",1785,75011,FR,Paris,Île-de-France,France,"[37 Rue Bréguet, 75011 Paris, France]",NaN,NaN
2,54b7ce9c498ef19600d79341,CrossFit XIII,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1588524588,False,14 rue de Vandrezanne,48.828197,2.356445,"[{'label': 'display', 'lat': 48.82819749353125...",3193,75013,FR,Paris,Île-de-France,France,"[14 rue de Vandrezanne, 75013 Paris, France]",NaN,NaN
3,4fa81d41e4b0d6f4be98c6bd,Reebok CrossFit Louvre,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1588524588,False,31 avenue de l'Opéra,48.867266,2.333469,"[{'label': 'display', 'lat': 48.86726626129496...",1766,75002,FR,Paris,Île-de-France,France,"[31 avenue de l'Opéra, 75002 Paris, France]",NaN,NaN
4,5de2304b18f0a7000753c237,Crossfit Lutèce 2,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1588524588,False,33 Rue de l'Église,48.844703,2.286984,"[{'label': 'display', 'lat': 48.844703, 'lng':...",4908,75015,FR,Paris,Île-de-France,France,"[33 Rue de l'Église, 75015 Paris, France]",NaN,NaN


In [43]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,CrossFit Lutèce,Gym,4 Rue Jules Simon,48.840888,2.292199,"[{'label': 'display', 'lat': 48.84088808067835...",4684,92015,FR,Paris,Île-de-France,France,"[4 Rue Jules Simon, 92015 Paris, France]",NaN,53e796e1498ed133e2e63026
1,CrossFit Original Addicts II,Gym,37 Rue Bréguet,48.858407,2.375695,"[{'label': 'display', 'lat': 48.85840655301172...",1785,75011,FR,Paris,Île-de-France,France,"[37 Rue Bréguet, 75011 Paris, France]",NaN,56a7c54b498e2517c6b9760b
2,CrossFit XIII,Gym / Fitness Center,14 rue de Vandrezanne,48.828197,2.356445,"[{'label': 'display', 'lat': 48.82819749353125...",3193,75013,FR,Paris,Île-de-France,France,"[14 rue de Vandrezanne, 75013 Paris, France]",NaN,54b7ce9c498ef19600d79341
3,Reebok CrossFit Louvre,Gym,31 avenue de l'Opéra,48.867266,2.333469,"[{'label': 'display', 'lat': 48.86726626129496...",1766,75002,FR,Paris,Île-de-France,France,"[31 avenue de l'Opéra, 75002 Paris, France]",NaN,4fa81d41e4b0d6f4be98c6bd
4,Crossfit Lutèce 2,Gym / Fitness Center,33 Rue de l'Église,48.844703,2.286984,"[{'label': 'display', 'lat': 48.844703, 'lng':...",4908,75015,FR,Paris,Île-de-France,France,"[33 Rue de l'Église, 75015 Paris, France]",NaN,5de2304b18f0a7000753c237
5,CrossFit Original Addicts,Gym,NaN,48.852923,2.397880,"[{'label': 'display', 'lat': 48.85292326371746...",3425,NaN,FR,NaN,NaN,France,[France],NaN,506083cee4b0351d980fe540
6,Reebok Crossfit Louvre 3,Gym / Fitness Center,24 Rue De Londres,48.877383,2.328346,"[{'label': 'display', 'lat': 48.877383, 'lng':...",2857,75009,FR,Paris,Île-de-France,France,"[24 Rue De Londres, 75009 Paris, France]",NaN,5dd636e883e1910007c80280
7,Silvaticus CrossFit,Gym,220 Boulevard Voltaire,48.852986,2.387952,"[{'label': 'display', 'lat': 48.852986, 'lng':...",2704,75011,FR,Paris,Île-de-France,France,"[220 Boulevard Voltaire, 75011 Paris, France]",NaN,57fe6afc498efbce0b9b0cda
8,CrossFit - Les Enfants de la Barre,Yoga Studio,156 Maurice Arnoux,48.811717,2.308539,"[{'label': 'display', 'lat': 48.81171699999999...",5912,92120,FR,Montrouge,Île-de-France,France,"[156 Maurice Arnoux, 92120 Montrouge, France]",NaN,589c28a001f07775462fe708
9,Crossfit Gavroche,Sports Club,29 rue des Peupliers,48.834295,2.257010,"[{'label': 'display', 'lat': 48.83429508957410...",7354,92100,FR,Boulogne-Billancourt,Île-de-France,France,"[29 rue des Peupliers, 92100 Boulogne-Billanco...",NaN,561cdc0c498e5051b09b6bd0


In [63]:
dataframe_filtered.dropna(subset = ["postalCode"], inplace=True)
dataframe_filtered


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,CrossFit Lutèce,Gym,4 Rue Jules Simon,48.840888,2.292199,"[{'label': 'display', 'lat': 48.84088808067835...",4684,92015,FR,Paris,Île-de-France,France,"[4 Rue Jules Simon, 92015 Paris, France]",NaN,53e796e1498ed133e2e63026
1,CrossFit Original Addicts II,Gym,37 Rue Bréguet,48.858407,2.375695,"[{'label': 'display', 'lat': 48.85840655301172...",1785,75011,FR,Paris,Île-de-France,France,"[37 Rue Bréguet, 75011 Paris, France]",NaN,56a7c54b498e2517c6b9760b
2,CrossFit XIII,Gym / Fitness Center,14 rue de Vandrezanne,48.828197,2.356445,"[{'label': 'display', 'lat': 48.82819749353125...",3193,75013,FR,Paris,Île-de-France,France,"[14 rue de Vandrezanne, 75013 Paris, France]",NaN,54b7ce9c498ef19600d79341
3,Reebok CrossFit Louvre,Gym,31 avenue de l'Opéra,48.867266,2.333469,"[{'label': 'display', 'lat': 48.86726626129496...",1766,75002,FR,Paris,Île-de-France,France,"[31 avenue de l'Opéra, 75002 Paris, France]",NaN,4fa81d41e4b0d6f4be98c6bd
4,Crossfit Lutèce 2,Gym / Fitness Center,33 Rue de l'Église,48.844703,2.286984,"[{'label': 'display', 'lat': 48.844703, 'lng':...",4908,75015,FR,Paris,Île-de-France,France,"[33 Rue de l'Église, 75015 Paris, France]",NaN,5de2304b18f0a7000753c237
6,Reebok Crossfit Louvre 3,Gym / Fitness Center,24 Rue De Londres,48.877383,2.328346,"[{'label': 'display', 'lat': 48.877383, 'lng':...",2857,75009,FR,Paris,Île-de-France,France,"[24 Rue De Londres, 75009 Paris, France]",NaN,5dd636e883e1910007c80280
7,Silvaticus CrossFit,Gym,220 Boulevard Voltaire,48.852986,2.387952,"[{'label': 'display', 'lat': 48.852986, 'lng':...",2704,75011,FR,Paris,Île-de-France,France,"[220 Boulevard Voltaire, 75011 Paris, France]",NaN,57fe6afc498efbce0b9b0cda
8,CrossFit - Les Enfants de la Barre,Yoga Studio,156 Maurice Arnoux,48.811717,2.308539,"[{'label': 'display', 'lat': 48.81171699999999...",5912,92120,FR,Montrouge,Île-de-France,France,"[156 Maurice Arnoux, 92120 Montrouge, France]",NaN,589c28a001f07775462fe708
9,Crossfit Gavroche,Sports Club,29 rue des Peupliers,48.834295,2.257010,"[{'label': 'display', 'lat': 48.83429508957410...",7354,92100,FR,Boulogne-Billancourt,Île-de-France,France,"[29 rue des Peupliers, 92100 Boulogne-Billanco...",NaN,561cdc0c498e5051b09b6bd0
12,Crossfit Tanka,Gym,418 Rue D'Etienne D'orves,48.914509,2.244954,"[{'label': 'display', 'lat': 48.914509, 'lng':...",10109,93700,FR,Colombes,Île-de-France,France,"[418 Rue D'Etienne D'orves, 93700 Colombes, Fr...",NaN,5618fbc4498e5b08a7ad6bc6


##### Step 7 : Extract the number of crossfit centers per "Arrondissement" and visualize it

In [95]:
dataframe_filtered.groupby('postalCode').postalCode.count()

postalCode
75002    1
75009    1
75011    2
75013    1
75015    1
92015    1
92100    2
92120    1
93400    1
93700    1
94200    1
Name: postalCode, dtype: int64

In [94]:
#Add the number of crossfit centers as a new column in the dataframe :

centers = [0,1,0,0,0,0,0,0,1,0,2,0,1,0,1,0,0,0,0,0]
df['Centers'] = centers
df


,Borough,Neighborhood,Density,Pricem2,Latitude,Longitude,Centers
0,75001,Louvre,9041,13681,48.8592,2.3417,0
1,75002,Bourse,21006,12274,48.8655,2.3426,1
2,75003,Temple,29956,12919,48.8637,2.3615,0
3,75004,Hotel de Ville,16966,13655,48.8601,2.3507,0
4,75005,Panthéon,23359,13107,48.8448,2.3471,0
5,75006,Luxembourg,19734,15416,48.8493,2.3300,0
6,75007,Palais Bourbon,13235,14650,48.8565,2.3210,0
7,75008,Elysée,9457,13160,48.8763,2.3183,0
8,75009,Opéra,27251,11623,48.8718,2.3399,1
9,75010,Saint Laurent,31754,10439,48.8709,2.3561,0


In [98]:
crossfit_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Paris

# add a red circle marker to represent Paris
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Paris',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(crossfit_map)

# add the Crossfit centers as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(crossfit_map)

# display map
crossfit_map

##### Step 8 : Decision criteria 

Lowest price per squared meter and neighborhood where no other center opened yet.

In [101]:
# Filter the data based on the price
df.sort_values(by='Pricem2', ascending=True)

,Borough,Neighborhood,Density,Pricem2,Latitude,Longitude,Centers
18,75019,Buttes Chaumont,27342,8895,48.8817,2.3822,0
19,75020,Ménilmontant,32702,9127,48.8646,2.3984,0
12,75013,Gobelins,25625,9969,48.8322,2.3561,1
17,75018,Montmartre,32875,9990,48.8925,2.3444,0
11,75012,Reuilly,22345,10041,48.8412,2.3876,0
9,75010,Saint Laurent,31754,10439,48.8709,2.3561,0
13,75014,Observatoire,24821,10702,48.8331,2.3264,0
10,75011,Popincourt,40827,10779,48.8574,2.3795,2
14,75015,Vaugirard,27712,10783,48.8412,2.3003,1
16,75017,Batignolles,29724,11204,48.8835,2.3219,0


#### Step 9 : Conclusion

Based on the data collected, here are my main observations :

* The lowest price per square meter is located in the 18 and 19 arrondissements.
* No crossfit center opened yet in these 2 neighborhoods.
* The population density is quite important in these 2 areas so no issue to build on new clients.

